在很多应用中，数据可能分布在多个文件或数据库中，抑或以某种不易于分析的格式进行排列。本章关注于对数据联合、连接以及重排列有用的工具。

首先，介绍pandas中的分层索引的概念，这个概念在这些操作中被广泛使用。然后深入介绍特定的数据操作。

In [1]:
import pandas as pd
import numpy as np

## 8.1　分层索引
分层索引是pandas的重要特性，允许你在一个轴向上拥有多个（两个或两个以上）索引层级。笼统地说，分层索引提供了一种在更低维度的形式中处理更高维度数据的方式。让我们以一个简单的例子开始，先创建一个Series，以列表的列表（或数组）作为索引：

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

data

a  1    0.451646
   2    1.462743
   3   -0.680673
b  1    1.307692
   3   -0.568636
c  1    0.229432
   2    0.039425
d  2   -0.308087
   3   -2.412937
dtype: float64

看到的是一个以`MultiIndex`作为索引的Series的美化视图。索引中的“间隙”表示“直接使用上面的标签”：

In [3]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

通过分层索引对象，也可以称为部分索引，允许简洁地选择出数据的子集：

In [4]:
data['b']

1    1.307692
3   -0.568636
dtype: float64

In [5]:
data['b':'c']

b  1    1.307692
   3   -0.568636
c  1    0.229432
   2    0.039425
dtype: float64

In [6]:
data.loc[['b', 'd']]

b  1    1.307692
   3   -0.568636
d  2   -0.308087
   3   -2.412937
dtype: float64

在“内部”层级中进行选择也是可以的：

In [7]:
data.loc[:, 2]

a    1.462743
c    0.039425
d   -0.308087
dtype: float64

分层索引在重塑数据和数组透视表等分组操作中扮演了重要角色。例如，可以使用`unstack`方法将数据在DataFrame中重新排列：

In [8]:
data.unstack()

,1,2,3
a,0.451646,1.462743,-0.680673
b,1.307692,NaN,-0.568636
c,0.229432,0.039425,NaN
d,NaN,-0.308087,-2.412937


`unstack`的反操作是`stack`：

In [9]:
data.unstack().stack()

a  1    0.451646
   2    1.462743
   3   -0.680673
b  1    1.307692
   3   -0.568636
c  1    0.229432
   2    0.039425
d  2   -0.308087
   3   -2.412937
dtype: float64

`stack`和`unstack`的细节将在后续部分进行探索。

在DataFrame中，每个轴都可以拥有分层索引：

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

分层的层级可以有名称（可以是字符串或Python对象）。如果层级有名称，这些名称会在控制台输出中显示：

In [11]:
frame.index.names = ['key1', 'key2']

frame.columns.names = ['state', 'color']

frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

> 请注意区分行标签中的索引名称'state'和'color'。

通过部分列索引，可以选出列中的组：

In [12]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [13]:
frame.loc[['a'], 'Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4

一个`MultiIndex`对象可以使用其自身的构造函数创建并复用。前面介绍的带有层级名称的DataFrame的列可以这样创建：

In [14]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                          names=['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           codes=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

### 8.1.1　重排序和层级排序
有时，需要重新排列轴上的层级顺序，或者按照特定层级的值对数据进行排序。`swaplevel`接收两个层级序号或层级名称，返回一个进行了层级变更的新对象（但是数据是不变的）：

In [15]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

另一方面，sort_index只能在单一层级上对数据进行排序。在进行层级变换时，使用sort_index以使得结果按照层级进行字典排序也很常见：

In [16]:
frame.sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [17]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [18]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

> 如果索引按照字典顺序从最外层开始排序，那么数据选择性能会更好——调用`sort_index`（level=0）或`sort_index`可以得到这样的结果。

### 8.1.2　按层级进行汇总统计
DataFrame和Series中很多描述性和汇总性统计有一个`level`选项，通过`level`选项可以指定想要在某个特定的轴上进行聚合。考虑上述示例中的DataFrame，可以按照层级在行或列上像下面这样进行聚合：

In [19]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [20]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

上面的例子使用了pandas的`groupby`机制，这个机制的细节将会在后续讨论。

### 8.1.3　使用DataFrame的列进行索引
通常不会使用DataFrame中一个或多个列作为行索引；反而可能想要将行索引移动到DataFrame的列中。下面是一个示例DataFrame：

In [21]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})

frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame的`set_index`函数会生成一个新的DataFrame，新的DataFrame使用一个或多个列作为索引：

In [22]:
frame2 = frame.set_index(['c', 'd'])

frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

默认情况下这些列会从DataFrame中移除，也可以将它们留在DataFrame中：

In [23]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

另一方面，`reset_index`是`set_index`的反操作，分层索引的索引层级会被移动到列中：

In [24]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2　联合与合并数据集
包含在pandas对象的数据可以通过多种方式联合在一起：

* `pandas.merge`根据一个或多个键将行进行连接。对于SQL或其他关系型数据库的用户来说，这种方式比较熟悉，它实现的是数据库的连接操作。

* `pandas.concat`使对象在轴向上进行黏合或“堆叠”。

* `combine_first`实例方法允许将重叠的数据拼接在一起，以使用一个对象中的值填充另一个对象中的缺失值。

### 8.2.1　数据库风格的DataFrame连接
合并或连接操作通过一个或多个键连接行来联合数据集。这些操作是关系型数据库的核心内容（例如基于SQL的数据库）。pandas中的`merge`函数主要用于将各种join操作算法运用在你的数据上：

In [25]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

print(df1)

print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
  key  data2
0   a      0
1   b      1
2   d      2


这是一个多对一连接的例子；df1的数据有多个行的标签为a和b，而df2在key列中每个值仅有一行。调用`merge`获得的对象：

In [26]:
# 默认是inner join
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


上述并没有指定在哪一列上进行连接。如果连接的键信息没有指定，`merge`会自动将重叠列名作为连接的键。但是，显式地指定连接键才是好的实现：

In [27]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


如果每个对象的列名是不同的，可以分别为它们指定列名：

In [28]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})

pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


结果中缺少'c'和'd'的值以及相关的数据。默认情况下，`merge`做的是`内连接`（'inner'join），结果中的键是两张表的交集。其他可选的选项有`'left'`、`'right'`和`'outer'`。`外连接`（outer join）是键的并集，联合了左连接和右连接的效果：

In [29]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


下表是对how选项的总结。

|  选项  |             描述             |
| ----- | --------------------------- |
| inner | 只对两张表都有的键的交集进行联合 |
| left  | 对所有左表的键进行联合          |
| right | 对所有右表的键进行联合          |
| outer | 对两张表都有的键的并集进行联合   |

尽管不是很直观，但多对多的合并有明确的行为。下面是一个例子：

In [30]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})

df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [31]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [32]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


多对多连接是行的笛卡尔积。由于在左边的DataFrame中有三个'b'行，而在右边有两行，因此在结果中有六个'b'行。连接方法仅影响结果中显示的不同键值：

In [33]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


使用多个键进行合并时，传入一个列名的列表：

In [34]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})

right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


决定哪些键联合出现在结果中，取决于合并方法的选择，把多个键看作一个元组数据来作为单个连接键使用（尽管实际上并不是以这种方法来实现的）。

> 当在进行`列-列`连接时，传递的DataFrame索引对象会被丢弃。

合并操作中最后一个要考虑的问题是如何处理重叠的列名。虽然你可以手动解决重叠问题（参见之前章节中的重命名轴标签的内容），但是merge有一个suffixes后缀选项，用于在左右两边DataFrame对象的重叠列名后指定需要添加的字符串：

In [35]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [36]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


下表是`merge`方法的参数参考。

| 参数  | 描述  |
|-|-|
| left        | 合并时操作中左边的 Dataframe                                                                                               |
| right       | 合并时操作中右边的 Dataframe                                                                                               |
| how         | 'inner'、'outer'、'left'、'right'之一;默认是'inner'                                                                     |
| on          | 需要连接的列名。<br>必须是在两边的 Data frame对象都有的列名,<br>并以`left`和`right`中的列名的交集作为连接键     |                              
| left on     | `left` Dataframe中用作连接键的列                                                                                             |
| right on    | `right` DataFrame中用作连接键的列                                                                                            |
| left index  | 使用left的行索引作为它的连接键(如果是 MultiIndex,则是多个键)                                                                  |
| right index | 使用right的行索引作为它的连接键(如果是 MultiIndex,则是多个键)                                                                 |
| sort        | 通过连接键按字母顺序对合并的数据进行排序;<br>在默认情况下为True(在大数据集上某些情况下禁用该功能可以获得更好的性能)  |
| suffixes    | 在重叠情况下,添加到列名后的字符串元组;默认是('x','y')<br>(例如如果待合并的 Data frame中都含有'data'列,那么结果中会出现'data'、'data_y') |
| copy        | 如果为 False,则在某些特殊情况下避免将数据复制到结果数据结构中;<br>默认情况下总是复制        |
| indicator   | 添加一个特殊的列 merge,指示每一行的来源;<br>值将根据每行中连接数据的来源分别为'left only',' right_ony'或'both'  |

### 8.2.2　根据索引合并
在某些情况下，DataFrame中用于合并的键是它的索引。在这种情况下，可以传递`left_index=True`或`right_index=True`（或者都传）来表示索引需要用来作为合并的键：

In [37]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})

right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [38]:
right1

,group_val
a,3.5
b,7.0


In [39]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


由于默认的合并方法是连接键相交，可以使用外连接来进行合并：

In [40]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


在多层索引数据的情况下，事情会更复杂，在索引上连接是一个隐式的多键合并：

In [41]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [42]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

这种情况下，必须以列表的方式指明合并所需多个列（请注意使用`how='outer'`处理重复的索引值）：

In [43]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [44]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


使用两边的索引进行合并也是可以的：

In [45]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [46]:
right2    

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [47]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame有一个方便的`join`实例方法，用于按照索引合并。该方法也可以用于合并多个索引相同或相似但没有重叠列的DataFrame对象。在之前的例子中，可以这样写：

In [48]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


由于一些历史原因（例如，一些非常早期版本的pandas），DataFrame的`join`方法进行连接键上的左连接，完全保留左边DataFrame的行索引。它还支持在调用DataFrame的某一列上连接传递的DataFrame的索引：

In [49]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


最后，对于一些简单索引-索引合并，可以向join方法传入一个DataFrame列表，这个方法可以替代将要介绍的使用更为通用的`concat`函数的方法：

In [50]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])

another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [51]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [52]:
left2.join([right2, another], how='outer', sort=False,)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6848: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


### 8.2.3　沿轴向连接
另一种数据组合操作可互换地称为拼接、绑定或堆叠。NumPy的`concatenate`函数可以在NumPy数组上实现该功能：

In [53]:
arr = np.arange(12).reshape((3, 4))

arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [54]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [55]:
np.concatenate([arr, arr])

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

在Series和DataFrame等pandas对象的上下文中，使用标记的轴可以进一步泛化数组连接。尤其是还有许多需要考虑的事情：

* 如果对象在其他轴上的索引不同，是否应该将不同的元素组合在这些轴上，还是只使用共享的值（交集）？

* 连接的数据块是否需要在结果对象中被识别？

* “连接轴”是否包含需要保存的数据？在许多情况下，DataFrame中的默认整数标签在连接期间最好丢弃。

pandas的`concat`函数提供了一种一致性的方式来解决以上问题。假设有三个索引不存在重叠的Series：

In [56]:
s1 = pd.Series([0, 1], index=['a', 'b'])

s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])

s3 = pd.Series([5, 6], index=['f', 'g'])

用列表中的这些对象调用`concat`方法会将值和索引粘在一起：

In [57]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

默认情况下，`concat`方法是沿着`axis=0`的轴向生效的，生成另一个Series。如果传递`axis=1`，返回的结果则是一个DataFrame（`axis=1`时是列）：

In [58]:
pd.concat([s1, s2, s3], axis=1, sort=False)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


在这个案例中另一个轴向上并没有重叠，可以看到排序后的索引合集（`'outer' join`外连接）。也可以传入`join='inner'`：

In [59]:
s4 = pd.concat([s1, s3])

s4

a    0
b    1
f    5
g    6
dtype: int64

In [60]:
pd.concat([s1, s4], axis=1, sort=False)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [61]:
pd.concat([s1, s4], axis=0)

a    0
b    1
a    0
b    1
f    5
g    6
dtype: int64

In [62]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


在这个例子中，由于`join='inner'`的选项，'f'和'g'标签消失了。

> 甚至可以使用`join_axes`来指定用于连接其他轴向的轴：

In [63]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


拼接在一起的各部分无法在结果中区分是一个潜在的问题。假设你想在连接轴向上创建一个多层索引，可以使用`keys`参数来实现：

In [64]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])

result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [65]:
result.index

MultiIndex(levels=[['one', 'two', 'three'], ['a', 'b', 'f', 'g']],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 2, 3]])

In [66]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


沿着轴向`axis=1`连接Series的时候，`keys`则成为DataFrame的列头：

In [67]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'], sort=True)

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


将相同的逻辑拓展到DataFrame对象：

In [68]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])

df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])

df1

,one,two
a,0,1
b,2,3
c,4,5


In [69]:
df2

,three,four
a,5,6
c,7,8


In [70]:
pd.concat([df1, df2], sort=False)

,one,two,three,four
a,0.0,1.0,NaN,NaN
b,2.0,3.0,NaN,NaN
c,4.0,5.0,NaN,NaN
a,NaN,NaN,5.0,6.0
c,NaN,NaN,7.0,8.0


In [71]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], sort=False)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

如果传递的是对象的字典而不是列表的话，则字典的键会用于`keys`选项：

In [72]:
pd.concat({'level1': df1, 'level2': df2}, axis=1, sort=False)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

还有一些额外的参数负责多层索引生成（见下表）。

| 参数 | 描述 |
| ---- | ---- |
|objs              | 需要连接的 pandas对象列表或字典;这是必选参数|
| axis              | 连接的轴向;默认是0(沿着行方向)|
| join              | 可以是'inner'或'outer'(默认是'outer');用于指定连接方式是内连接(inner)还是外连接(outer)|
| join_axes         | 用于指定其他n-1轴的特定索引,可以替代内/外连接的逻辑|
| keys              | 与要连接的对象关联的值,沿着连接轴形成分层索引;<br>可以是任意值的列表或数组,也可以是元组的数组,也可以是数组的列表<br>(如果向levels参数传入多层数组) |
| levels            | 在键值传递时,该参数用于指定多层索引的层级|
| names             | 如果传入了keys和/或levels参数,该参数用于多层索引的层级名称|
| verify_intergrity | 检查连接对象中的新轴是否重复,如果是,则引发异常;默认(False)允许重复|
| ignore_index      | 不沿着连接轴保留索引,而产生一段新的(长度为total_length)索引|

例如，可以使用`names`参数命名生成的轴层级：

In [73]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'], sort=True)

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

最后需要考虑的是行索引中不包含任何相关数据的DataFrame：

In [74]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])

df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

df1

,a,b,c,d
0,-0.009373,0.093308,0.443906,-2.019238
1,0.695774,0.583176,-0.893969,-0.661266
2,-0.149492,-0.333352,1.326730,1.112550


In [75]:
df2

,b,d,a
0,0.382933,0.555190,-0.806608
1,0.385983,0.354565,0.684034


在这个示例中，传入`ignore_index=True`：

In [76]:
pd.concat([df1, df2], ignore_index=True, sort=False)

,a,b,c,d
0,-0.009373,0.093308,0.443906,-2.019238
1,0.695774,0.583176,-0.893969,-0.661266
2,-0.149492,-0.333352,1.326730,1.112550
3,-0.806608,0.382933,NaN,0.555190
4,0.684034,0.385983,NaN,0.354565


### 8.2.4　联合重叠数据
还有另一个数据联合场景，既不是合并操作，也不是连接操作。可能有两个数据集，这两个数据集的索引全部或部分重叠。作为一个示例，考虑NumPy的`where`函数，这个函数可以进行面向数组的`if-else`等价操作：

In [77]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])

b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=['a', 'b', 'c', 'd', 'e', 'f'])

a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [78]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [79]:
# 与index无关，与数值顺序有关
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

Series有一个`combine_first`方法，该方法可以等价于下面这种使用pandas常见数据对齐逻辑的轴向操作：

In [80]:
# 与index有关
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

在DataFrame中，`combine_first`逐列做相同的操作，因此可以认为它是根据传入的对象来“修补”调用对象的缺失值：

In [81]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})

df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [82]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [83]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [84]:
df2.combine_first(df1)

,a,b,c
0,5.0,NaN,2.0
1,4.0,3.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3　重塑和透视
重排列表格型数据有多种基础操作。这些操作被称为重塑或透视。

### 8.3.1　使用多层索引进行重塑
多层索引在DataFrame中提供了一种一致性方式用于重排列数据。以下是两个基础操作：

* statck（堆叠）：该操作会“旋转”或将列中的数据透视到行。

* unstack（拆堆）：该操作会将行中的数据透视到列。

通过例子来说明这些操作。考虑一个带有字符串数组作为行和列索引的小型DataFrame：

In [85]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                                     name='number'))

data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


在这份数据上使用`stack`方法会将列透视到行，产生一个新的Series：

In [86]:
result = data.stack()

result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

从一个多层索引序列中，可以使用`unstack`方法将数据重排列后放入一个DataFrame中：

In [87]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


默认情况下，最内层是已拆堆的（与`stack`方法一样）。可以传入一个层级序号或名称来拆分一个不同的层级：

In [88]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [89]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


如果层级中的所有值并未包含于每个子分组中时，拆分可能会引入缺失值：

In [90]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])

s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

data2 = pd.concat([s1, s2], keys=['one', 'two'])

data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [91]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


默认情况下，堆叠会过滤出缺失值，因此堆叠拆堆的操作是可逆的：

In [92]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [93]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

当在DataFrame中拆堆时，被拆堆的层级会变为结果中最低的层级：

In [94]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))

df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [95]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

在调用`stack`方法时，可以指明需要堆叠的轴向名称：

In [96]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### 8.3.2　将“长”透视为“宽”
在数据库和CSV中存储多时间序列的方式就是所谓的长格式或堆叠格式。载入一些示例数据，然后做少量的时间序列规整和其他的数据清洗操作：

In [97]:
data = pd.read_csv('examples/macrodata.csv')

data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [98]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.head()

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2


In [99]:
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})

后续会更深入地讲解`PeriodIndex`。简单地说，`PeriodIndex`将year和quarter等列进行联合并生成了一种时间间隔类型：

现在，ldata看起来如下：

In [100]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


这种数据即所谓的多时间序列的长格式，或称为具有两个或更多个键的其他观测数据（这里，键是date和item）。表中的每一行表示一个时间点上的单个观测值。

数据通常以这种方式存储在关系型数据库中，比如MySQL，因为固定模式（列名称和数据类型）允许item列中不同值的数量随着数据被添加到表中而改变。在之前的例子中，date和item通常是主键（使用关系型数据库的说法），提供了关系完整性和更简单的连接。在某些情况下，处理这种格式的数据更为困难，可能更倾向于获取一个按date列时间戳索引的且每个不同的item独立一列的DataFrame。DataFrame的`pivot`方法就是进行这种转换的：

In [101]:
pivoted = ldata.pivot('date', 'item', 'value')

pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


传递的前两个值是分别用作行和列索引的列，然后是可选的数值列以填充DataFrame。假设有两个数值列，想同时进行重塑：

In [102]:
ldata['value2'] = np.random.randn(len(ldata))

ldata.head()

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-1.460561
1,1959-03-31 23:59:59.999999999,infl,0.000,0.813255
2,1959-03-31 23:59:59.999999999,unemp,5.800,1.089398
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.713697
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.463863


如果遗漏最后一个参数，会得到一个含有多层列的DataFrame：

In [103]:
pivoted = ldata.pivot('date', 'item')

pivoted.head()

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.813255 -1.460561   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.463863 -0.713697   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  1.404438 -0.098607   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.666305  0.203987   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.250733  0.748653   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.089398  
1959-06-30 23:59:59.999999999 -0.139772  
1959-09-30 23:59:59.999999999 -3.768260  
1959-12-31 23:59:59.999999999  1.585640  
1960-03-31 23:59:59.999999999  0.558553

In [104]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


请注意，`pivot`方法等价于使用`set_index`创建分层索引，然后调用`unstack`：

In [105]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')

unstacked.head()

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.813255 -1.460561   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.463863 -0.713697   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  1.404438 -0.098607   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.666305  0.203987   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.250733  0.748653   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.089398  
1959-06-30 23:59:59.999999999 -0.139772  
1959-09-30 23:59:59.999999999 -3.768260  
1959-12-31 23:59:59.999999999  1.585640  
1960-03-31 23:59:59.999999999  0.558553

### 8.3.3　将“宽”透视为“长”
在DataFrame中，`pivot`方法的反操作是`pandas.melt`。与将一列变换为新的Data Frame中的多列不同，它将多列合并成一列，产生一个新的DataFrame，其长度比输入更长。看下面这个例子：

In [106]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})

df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


'key'列可以作为分组指标，其他列均为数据值。当使用`pandas.melt`时，必须指明哪些列是分组指标（如果有的话）。此处，使用'key'作为唯一的分组指标：

In [107]:
melted = pd.melt(df, id_vars=['key'])

melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


使用`pivot`方法，可以将数据重塑回原先的布局：

In [108]:
reshaped = melted.pivot('key', 'variable', 'value')

reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


由于`pivot`的结果根据作为行标签的列生成了索引，可能会想要使用`reset_index`来将数据回移一列：

In [109]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


可以指定列的子集作为值列：

In [110]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


`pandas.melt`的使用也可以无须任何分组指标：

In [111]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [112]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
